<a href="https://colab.research.google.com/github/VictoriousMango/Data-Science/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importing Libraries
import pickle 
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'keras'

In [ ]:
with open("train_qa.txt", "rb") as fp:
  train_data = pickle.load(fp)

In [ ]:
with open("test_qa.txt", "rb") as fp:
  test_data = pickle.load(fp)

In [ ]:
' '.join(train_data[0][0]) # Story
' '.join(train_data[0][1]) # Question
' '.join(train_data[0][2]) # Answer

# Set up Vocabulary
vocab = set()
all_data = test_data + train_data

for story, question, answer in all_data:
  vocab = vocab.union(set(story))
  vocab = vocab.union(set(question))
  vocab = vocab.union(set(answer))

vocab_len = len(vocab) + 1
max_ques_len = max([len(data[0]) for data in all_data])
max_ques_len = max([len(data[1]) for data in all_data])

In [2]:
# Vecotrizeing the Data
tokenizer = Tokenizer(filters = [])
tokenizer.fit_on_texts(vocab)
tokenizer.word_index
train_story_text = []
train_question_text = []
train_answers_text = []

for story, question, answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)
  train_answers_text.append(answer)

train_story_seq = tokenizer.texts_to_sequences(train_story_text)


NameError: ignored

In [ ]:
def vectorize_stories(data, word_index = tokenizer.word_index, max_story_len = max_story_len, max_question_len = max_question_len):
    X  = [] # Stories
    Xq = [] # Query/Question Answer
    Y  = [] # Correct Answer
    
    for story, query, answer in data:
        x  = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]
        y  = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
        
        return (pad_sequences(X, maxlen = max_story_len),
               pad_sequences(Xq, maxlen = max_ques_len),
               np.array(Y))

In [ ]:
input_sequence = Input((max_story_len,))
question = Input((max_ques_len,))
# Input Encoder m
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim = vocab_len, output_dim = 64))
input_encoder_m.add(Dropout(0.3))

In [ ]:
# Input Encoder c
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim = vocab_len, output_dim = max_ques_len))
input_encoder_c.add(Dropout(0.3))

In [ ]:
# Question Encoder
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim = vocab_len, output_dim = 64, input_length = max_ques_len))
question_encoder.add(Dropout(0.3))

In [ ]:
# Encode the Sequence
input_encoded_m  = input_encoder_m(input_sequence)
input_encoded_c  = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [1]:
match = dot([input_encoded_m, question_encoded], axes = (2, 2))
match = Activation('softmax')(match)

response = add([match, input_encoded_c])
response = Permute((2, 1))(response)

answer = concatenate([response, question_encoded])
answer = LSTM(32)(answer)
answer = Dropout(0.5)(answer)
answer = Dense(vocab_len)(answer)
answer = Activation('softmax')(answer)

NameError: name 'dot' is not defined

In [ ]:
model = Model([input_sequence, question], answer)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()
history = model.fit([input_train, queries_train], answer_train, batch_size = 32, epochs = 18, validation_data = ([input_test, queries_test], answers_test))

In [ ]:
print(history.history.keys())

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.xlabel("epochs")
plt.ylabel("Accuracy")

In [ ]:
model.save("chatbot_model")         # Save
model.load_weights("chatbot_model") # Evaluation on the Test Set

In [ ]:
# Testing the Chatbot
pred_results = model.predict(([input_test, queries_test]))

val_max = np.argmax(pred_results[37])

for key, value in tokenizer.word_index.items():
    if value == val_max:
        k = key
        
print("Predicted answer is : ", k)
print("Probability of Certainity : ", pred_results[37][val_max])